In [ ]:
pip install mysqlclient mysql-connector-python pymysql SQLAlchemy pandas nltk numpy

In [ ]:
import mysql.connector as sql
import pandas as pd

db_connection = sql.connect(host='localhost', database='cms', user='root', password='')
db_cursor = db_connection.cursor()
db_cursor.execute('SELECT * FROM tblcomplaints')

table_rows = db_cursor.fetchall()

df = pd.DataFrame(table_rows)

In [ ]:
from sqlalchemy import create_engine
import pymysql

db_connection_str = 'mysql+pymysql://root:@localhost/cms'
db_connection = create_engine(db_connection_str)

df = pd.read_sql('SELECT * FROM tblcomplaints', con=db_connection)

In [ ]:
df['days_compound'] = df.apply(lambda row: ((row.noc/10)*-1)+1, axis=1)

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
df['score2'] = df['complaintDetails'].apply(lambda review: sid.polarity_scores(review))

In [ ]:
df['compound']  = df['score2'].apply(lambda score_dict: score_dict['compound'])

In [ ]:
df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

In [ ]:
df['final_compound'] = df.apply(lambda row: row.days_compound + row.compound, axis=1)

In [ ]:
df = df.sort_values(by ='final_compound' )

In [ ]:
new = df[['complaintNumber', 'userId', 'category','subcategory','complaintType','state','noc','complaintDetails','complaintFile','regDate','status','lastUpdationDate','days_compound','compound','comp_score','final_compound']].copy()

In [ ]:
sqlEngine       = create_engine('mysql+pymysql://root:@localhost/test', pool_recycle=3600)

dbConnection    = sqlEngine.connect()
tableName="abc"
try:

    frame           = new.to_sql(tableName, dbConnection, if_exists='replace');

except ValueError as vx:

    print(vx)

except Exception as ex:   

    print(ex)

else:

    print("Table %s created successfully."%tableName);   

finally:

    dbConnection.close()